In [1]:
import numpy as np
import scipy.signal
import os
import random

In [15]:
def normalize_signal(signal):
    signal = signal.copy()
    non_zero_mask = np.any(signal != 0, axis=1)  # Ignoriraj prazne vrstice
    if np.any(non_zero_mask):
        non_zero_signal = signal[non_zero_mask]
        min_val = non_zero_signal.min()
        max_val = non_zero_signal.max()
        if max_val > min_val:
            signal[non_zero_mask] = (non_zero_signal - min_val) / (max_val - min_val)
    return signal

def add_noise(signal, noise_level=0.01):
    # Dodam random noise, ustvari "jitter" efekt
    noise = np.random.normal(0, noise_level, signal.shape)
    signal = signal.copy()
    non_zero_mask = np.any(signal != 0, axis=1)
    signal[non_zero_mask] += noise[non_zero_mask]
    return signal

def frame_shift(signal, max_shift=10):
    # Zamaknem frame, ustvari efekt zamujene/predčasne geste
    shift = np.random.randint(-max_shift, max_shift + 1)
    signal = signal.copy()
    if shift > 0:
        signal[:-shift] = signal[shift:]
        signal[-shift:] = 0
    elif shift < 0:
        shift = abs(shift)
        signal[shift:] = signal[:-shift]
        signal[:shift] = 0 
    return signal

def smoothing(signal, window_size=5):
    # Hann window za glajenje podatkov/gest
    window = scipy.signal.windows.hann(window_size)
    signal = signal.copy()
    non_zero_mask = np.any(signal != 0, axis=1)
    frames = np.where(non_zero_mask)[0]
    if len(frames) > window_size:
        for i in range(signal.shape[1]):
            signal[frames, i] = np.convolve(signal[frames, i], window / window.sum(), mode='same')
    return signal

In [16]:
def transform(signal):
    signal = signal.copy()
    signal = normalize_signal(signal)

    # Randomly izbere augmentacijo
    if random.random() < 0.7:
        signal = add_noise(signal)

    if random.random() < 0.7:
        signal = frame_shift(signal)

    if random.random() < 0.7:
        signal = smoothing(signal)

    signal = normalize_signal(signal)  # Ponovna normalizacija

    return {'data': signal}

In [ ]:
input_file = "Podatki/desna_odprto_dol/desna_odprto_dol_1.npy"  # Vhodni 3D signal
signal = np.load(input_file)
print("Originalni signal:", signal.shape)

base_name = os.path.splitext(os.path.basename(input_file))[0]

output_dir = os.path.join("Augmented_Podatki", base_name)
os.makedirs(output_dir, exist_ok=True)

np.random.seed(42)

# Generiranje 10 novih primerkov
for i in range(10):
    augmented = transform(signal)
    augmented_signal = augmented['data']

    output_path = os.path.join(output_dir, f"augmented_signal_{i}.npy")
    np.save(output_path, augmented_signal)

    print(f"Saved: {output_path}, shape: {augmented_signal.shape}")